# 4. Softmax Regression
다중분류 문제를 해결하기 위해 사용하는 회귀방식으로 Softmax 함수를 이용하여 문제를 해결한다.

## 4-1 One-hot encoding

벡터로 표현할 때, 선택지의 개수만큼 크기를 가지며, 각 선택지에 해당되는 원소를 1로 설정한다. 나머지는 0으로 설정

In [7]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

In [9]:
z = torch.FloatTensor([[1, 2, 3]])
print(z)

tensor([[1., 2., 3.]])


In [14]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.0900, 0.2447, 0.6652]])


In [15]:
hypothesis.sum()

tensor(1.)

In [16]:
z = torch.rand(3, 5, requires_grad=True)

In [17]:
print(z)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)


In [22]:
hypothesis = F.softmax(z, dim=1)

In [23]:
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [26]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 1, 4])


In [27]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [32]:
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

In [34]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6160, grad_fn=<MeanBackward0>)


In [35]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [36]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [37]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.6160, grad_fn=<NllLossBackward0>)

In [38]:
F.cross_entropy(z, y)

tensor(1.6160, grad_fn=<NllLossBackward0>)

##4-2 implement Softmax Regression

### Implement in Low-level

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [40]:
torch.manual_seed(1)

In [41]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [42]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1, y_train.unsqueeze(1),1)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [43]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W,b], lr= 0.1)

In [44]:
res = x_train.matmul(W) + b
print(res.shape)

torch.Size([8, 3])


we need to compute softmax in dim=1 since we are finding the highest prob among 3 classes

In [46]:
num_epochs = 1000
for epoch in range(num_epochs + 1):
  hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)

  cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### Implement in High-level

In [48]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W,b], lr=0.1)

num_epochs = 1000
for epoch in range(num_epochs+1):
  z = x_train.matmul(W) + b
  cost = F.cross_entropy(z, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### Implement Softmax regression using `nn.Module`

In [49]:
model = nn.Linear(4, 3)

In [50]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

num_epochs=1000

for epoch in range(num_epochs+1):
  pred = model(x_train)
  cost = F.cross_entropy(pred, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


In [53]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)

  def forward(self, x):
    return self.linear(x)

In [54]:
model = SoftmaxClassifierModel()

In [55]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

num_epochs=1000
for epoch in range(num_epochs+1):
  pred = model(x_train)
  cost = F.cross_entropy(pred, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
